In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange


import flygym.common
import flygym.mujoco
import flygym.mujoco.preprogrammed

from flygym.mujoco.arena.base import BaseArena, FlatTerrain
from flygym.mujoco import Parameters

from nmf_project import (
    NeuromechflyProject,
    all_actuated_dof,)


from nmf_grooming import (
    NeuromechflyGrooming,
    all_groom_dofs,
    load_grooming_data,
    plot_state_and_contacts,
)
from scipy.signal import find_peaks, medfilt
from IPython.display import Video

from flygym.mujoco.examples.rule_based_controller import PreprogrammedSteps

In [2]:
run_time = 1

sim_params = flygym.mujoco.Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.2,
    enable_adhesion=True,
    draw_adhesion=True,
)

actuated_joints = flygym.mujoco.preprogrammed.all_leg_dofs

In [3]:
target_num_steps = int(run_time / sim_params.timestep) # 10'000


In [4]:
nmf = flygym.mujoco.NeuroMechFly(Parameters(
    enable_adhesion=True, 
    draw_adhesion=True,
    render_camera="Animat/camera_left"))

preprogrammed_steps = PreprogrammedSteps()

swing_periods = preprogrammed_steps.swing_period

legs = preprogrammed_steps.legs

actuated_joints = preprogrammed_steps.dofs_per_leg

standing_action = []

for leg in legs:
    if leg.endswith("M"):
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1]))
    else:
        standing_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))

standing_action = {'joints': standing_action, "adhesion": np.zeros(len(legs))}

for i in range(int(0.2//nmf.timestep)):
    nmf.step(standing_action)



In [5]:
nmf.reset()
foreleg_ids = np.zeros(target_num_steps)
middle_stance_ids = np.linspace(swing_periods["RM"][1], -1/4*np.pi, target_num_steps)
hind_swings_ids = np.zeros(target_num_steps)

adhesion_action = np.array([1.0 if leg.endswith("F") else 0.0 for leg in legs])
all_joint_angles = []

for i in trange(target_num_steps):
    joint_angles = []
    for leg in legs:
        if leg.endswith("H"):
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, hind_swings_ids[i]))
        elif leg.endswith("M"):
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, middle_stance_ids[i]))
        else:
            joint_angles.extend(preprogrammed_steps.get_joint_angles(leg, foreleg_ids[i]))
    all_joint_angles.append(joint_angles.copy())

    action = {'joints': np.array(joint_angles), "adhesion": adhesion_action}
    nmf.step(action)
    nmf.render()
nmf.save_video("./outputs/b.mp4")

last_joint_angles = all_joint_angles[-1]


100%|██████████| 10000/10000 [00:40<00:00, 245.59it/s]


In [ ]:
plt.plot(all_joint_angles)

In [ ]:
plt.plot(last_joint_angles)
print(len(last_joint_angles))
print(last_joint_angles)

In [ ]:
Video("./outputs/b.mp4")

#### Try to move abdoment but didn't work out

In [ ]:
# Try to move abdomen, didn't work out

# from example given in miniproject
# abdomen_control = np.sin(
#     2 * 2 * np.pi * np.linspace(0, 0.5, nbre_steps) / 0.5
# )
# abdomen_control = abdomen_control * np.pi / 5 + np.pi / 10
# abdomen_control = np.tile(abdomen_control, (5, 1)).T # shape (nbre_steps, 5)


# for i in range(len(all_actuated_dof)):
#     if i == 42: # LPedicel
#         all_joint_pos[:,i] = np.linspace(0,0, nbre_steps)
#     elif i == 43: # LPedicel_yaw
#         all_joint_pos[:,i] = np.linspace(0, 0, nbre_steps)
#     elif i == 44: # RPedicel
#         all_joint_pos[:,i] = np.linspace(0, 0, nbre_steps)
#     elif i == 45: # RPedicel_yaw
#         all_joint_pos[:,i] = np.linspace(0, 0, nbre_steps)
#     elif i == 46: # A1A2
#         all_joint_pos[:,i] = np.linspace(0, np.pi/6, nbre_steps)
#     elif i == 47: # A3
#         all_joint_pos[:,i] = np.linspace(0, np.pi/6, nbre_steps)
#     elif i == 48 : # A4
#         all_joint_pos[:,i] = np.linspace(0, np.pi/6, nbre_steps)
#     elif i == 49 : # A5
#         all_joint_pos[:,i] = np.linspace(0, np.pi/6, nbre_steps)
#     elif i == 50 : # A6
#         all_joint_pos[:,i] = np.linspace(0, np.pi/6, nbre_steps)
#     else:
    # all_joint_pos[:,i] = np.linspace(last_joint_angles[i], last_joint_angles[i], nbre_steps)


# pedicel_arr = np.zeros((nbre_steps, 4))
# abd_arr = 0.25*np.ones((nbre_steps, 5))
# all_joint_pos = np.concatenate([all_joint_pos,pedicel_arr, abd_arr], axis=1)
# print(all_joint_pos.shape)
# leg_action = []
# for leg in preprogrammed_steps.legs:
#     # fly is standing
#     leg_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))
# adhesion_action = np.array([0.0 if leg.endswith("H") else 1.0 for leg in legs])

#### Move Abdomen up for Hindleg grooming

In [6]:
# Move Abdomen Up
nbre_steps = 5000

nmf_act_abd = NeuromechflyProject(Parameters(timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.2,
    enable_adhesion=True,
    draw_adhesion=True,
    render_camera="Animat/camera_left",))

# Actuate abdomen --> move up to reach horizontal position
abdomen_control = 0.25*np.ones((nbre_steps, 4)) # shape (nbre_steps, 4)
last_abd_ctrl = 0.2*np.ones((nbre_steps,1)) # shape (nbre_steps, 1)
abdomen_control = np.concatenate([abdomen_control, last_abd_ctrl], axis=1) # shape (nbre_steps, 5)

# Keep last position of the abdomen
last_abd_pos = abdomen_control[-1,:] # shape (1,5)

# Keep all other joints at last position during the whole movement  
joint_pos = np.tile(last_joint_angles, (nbre_steps, 1)) # shape (nbre_steps, 42)

# Add abdomen control to all joint positions
all_joint_pos = np.concatenate([joint_pos, abdomen_control], axis=1) # shape (nbre_steps, 47)

# Reset the simulation
nmf_act_abd.reset()


for i in trange(nbre_steps):
    joint_angles = all_joint_pos[i,:]
    action = {'joints': joint_angles, "adhesion": adhesion_action}
    nmf_act_abd.step(action)
    nmf_act_abd.render()

nmf_act_abd.save_video("./outputs/up_abd.mp4")

100%|██████████| 5000/5000 [00:19<00:00, 256.59it/s]


In [ ]:
Video("./outputs/up_abd.mp4")

##### Plot leg action / last joint to see what was what

In [ ]:
leg_action = []
for leg in preprogrammed_steps.legs:
    # fly is standing
    leg_action.extend(preprogrammed_steps.get_joint_angles(leg, 0.0))
adhesion_action = np.array([0.0 if leg.endswith("H") else 1.0 for leg in legs])

plt.plot(leg_action)
plt.plot(all_joint_pos[0,:], 'r')
plt.plot(last_joint_angles, 'g')

##### Hindleg grooming

In [7]:
# Joint angles according to indices
LH_COXA = 14
LH_COXA_ROLL = 15
LH_COXA_YAW = 16
LH_FEMUR = 17
LH_FEMUR_ROLL = 18
LH_TIBIA = 19
LH_TARSUS = 20

RH_COXA = 35
RH_COXA_ROLL = 36
RH_COXA_YAW = 37
RH_FEMUR = 38
RH_FEMUR_ROLL = 39
RH_TIBIA = 40
RH_TARSUS = 41

In [ ]:
nmf_act_abd.reset()

# Move Hindleg up
nbre_steps = 5000

# Keep abdomen up
abdomen_control = np.tile(last_abd_pos,(nbre_steps,1)) # shape (nbre_steps, 5)

# Keep last position of legs
joint_pos = np.tile(last_joint_angles, (nbre_steps, 1)) # shape (nbre_steps, 42)

                                #### Move hindlegs up ####
# Left hindleg
joint_pos[:,LH_COXA] = np.linspace(last_joint_angles[LH_COXA], last_joint_angles[LH_COXA]+last_joint_angles[LH_COXA]/2, nbre_steps) 
joint_pos[:,LH_COXA_YAW] = np.linspace(last_joint_angles[LH_COXA_YAW], last_joint_angles[LH_COXA_YAW]-last_joint_angles[LH_COXA_YAW]/2, nbre_steps)
joint_pos[:,LH_TIBIA] = np.linspace(last_joint_angles[LH_TIBIA], last_joint_angles[LH_TIBIA]-last_joint_angles[LH_TIBIA]/3, nbre_steps)
#joint_pos[:,LH_TARSUS] = np.linspace(last_joint_angles[LH_TARSUS], -np.pi/12, nbre_steps)

# Right hindleg
joint_pos[:,RH_COXA] = np.linspace(last_joint_angles[RH_COXA], last_joint_angles[RH_COXA]+last_joint_angles[RH_COXA]/2, nbre_steps)
joint_pos[:,RH_COXA_YAW] = np.linspace(last_joint_angles[RH_COXA_YAW], last_joint_angles[RH_COXA_YAW]-last_joint_angles[RH_COXA_YAW]/2, nbre_steps)
joint_pos[:,RH_TIBIA] = np.linspace(last_joint_angles[RH_TIBIA], last_joint_angles[RH_TIBIA]-last_joint_angles[RH_TIBIA]/2, nbre_steps) 
#joint_pos[:,RH_TARSUS] = np.linspace(last_joint_angles[RH_TARSUS], -np.pi/6, nbre_steps)


# Add abdomen control to all joint positions
all_joint_angles = np.concatenate([joint_pos, abdomen_control], axis=1) # shape (nbre_steps, 47)

adhesion_action = np.array([0.0 if leg.endswith("H") else 1.0 for leg in legs])

for i in trange(nbre_steps):
    joint_angles = all_joint_angles[i,:]
    action = {'joints': joint_angles, "adhesion": adhesion_action}
    nmf_act_abd.step(action)
    nmf_act_abd.render()

nmf_act_abd.save_video("./outputs/up_hind.mp4")

In [ ]:
Video("./outputs/up_hind.mp4")

In [11]:
target_num_steps = 5000
all_joint_angles = np.ones((target_num_steps, len(all_joint_angles[0]))) # shape (nbre_steps, 42)
print(all_joint_angles.shape)

for i in range(len(all_joint_angles[0])):
    if i == 5: # LF tibia
        all_joint_angles[:,i] = np.linspace(last_joint_angles[5], 1/3*np.pi, target_num_steps)
    elif i == 6 : # LF tarsus
        all_joint_angles[:,i] = np.linspace(last_joint_angles[6], 0, target_num_steps)
    elif i == 12: # LM tibia
        all_joint_angles[:,i] = np.linspace(last_joint_angles[12], 1/3*np.pi, target_num_steps)
    elif i == 13: # LM tarsus
        all_joint_angles[:,i] = np.linspace(last_joint_angles[13], 0, target_num_steps)
    elif i == 26: # RF tibia
        all_joint_angles[:,i] = np.linspace(last_joint_angles[26], 1/3*np.pi, target_num_steps)
    elif i == 27: # RF tarsus
        all_joint_angles[:,i] = np.linspace(last_joint_angles[27], 0, target_num_steps)
    elif i == 33: # RM tibia
        all_joint_angles[:,i] = np.linspace(last_joint_angles[33], 1/3*np.pi, target_num_steps)
    elif i == 34: # RM tarsus
        all_joint_angles[:,i] = np.linspace(last_joint_angles[34], 0, target_num_steps)
    else:
        all_joint_angles[:,i] = np.linspace(last_joint_angles[i], last_joint_angles[i], target_num_steps)

all_joint_angles = np.concatenate([all_joint_angles, abdomen_control], axis=1)

adhesion_action = np.array([0.0 if leg.endswith("H") else 1.0 for leg in legs])

for i in range(target_num_steps):
    joint_angles = all_joint_angles[i,:]
    action = {'joints': joint_angles, "adhesion": adhesion_action}
    nmf_act_abd.step(action)
    nmf_act_abd.render()
nmf_act_abd.save_video("./outputs/c.mp4")

(5000, 42)


In [ ]:
nmf_act_abd.reset()
all_joint_angles = np.ones((target_num_steps, len(all_joint_angles[0])))

if i == 9: # LH Tarsus
    all_joint_angles[:,i] = np.linspace(last_joint_angles[9], 0, target_num_steps)
elif i == 41: # RH Tarsus
    all_joint_angles[:,i] = np.linspace(last_joint_angles[41], 0, target_num_steps)

for i in trange(target_num_steps):
    joint_angles = all_joint_angles[i,:]
    action = {"joints": joint_angles, "adhesion" : np.array([1,1,0,1,1,0])}
    #print("prog_action", action,'\n')
    nmf_act_abd.step(action)
    nmf_act_abd.render()
nmf_act_abd.save_video("./outputs/c.mp4")

In [ ]:
Video("./outputs/c.mp4")

In [ ]:
# plot hindleg joint positions
plt.figure(figsize=(15, 5))
plt.plot(all_joint_angles[:,8:9], label=["LH Tibia","LH Tarsus"])
plt.plot(all_joint_angles[:,14:15], label=["RH Tibia","RH Tarsus"])
plt.title("Hindleg joint positions")
plt.xlabel("Time step")
plt.ylabel("Joint angle")
plt.legend()
plt.show()

In [ ]:
observ = nmf.get_observation()
print(observ["end_effectors"])

#### Load grooming position of frontleg

In [ ]:
grooming_module_path = Path("./data/grooming_modules_provided_slow.pkl")
with open(grooming_module_path, "rb") as f:
    grooming_modules = pickle.load(f)

timestep_groom = grooming_modules["timestep"]
target_num_steps_groom = int(1.0 / timestep_groom) # 10'000
target_joint_angles_front = grooming_modules["foreleg"][:, :target_num_steps]


In [ ]:
# all_joint_angles.shape() = (10000, 42)
# my all_joint_angles_groom_shape() = (558, 21)

# need to map the 21 joints to the 42 joints
target_joint_angles_front = target_joint_angles_front.T # (558, 21)
target_joint_angles_front = np.tile(target_joint_angles_front, (18, 1)) # (10044, 21)

joint_LHCoxa = target_joint_angles_front[:,3]

In [ ]:
print(joint_LHCoxa.shape)
print(joint_LHCoxa[:target_num_steps].shape)
test = np.ones((target_num_steps, len(all_joint_angles[0])))
print(test.shape)
print(target_num_steps)

#### Map the grooming position of the leg to the corresponding body part.

In [ ]:
joint_LHCoxa = target_joint_angles_front[:,3]
joint_LHCoxa_roll = target_joint_angles_front[:,4]
joint_LHCoxa_yaw = target_joint_angles_front[:,5]
joint_LHFemur = target_joint_angles_front[:,6]
joint_LHFemur_roll = target_joint_angles_front[:,7]
joint_LHTibia = target_joint_angles_front[:,8]
joint_LHTarsus = target_joint_angles_front[:,9]
joint_RHCoxa = target_joint_angles_front[:,10]
joint_RHCoxa_roll = target_joint_angles_front[:,11]
joint_RHCoxa_yaw = target_joint_angles_front[:,12]
joint_RHFemur = target_joint_angles_front[:,13]
joint_RHFemur_roll = target_joint_angles_front[:,14]
joint_RHTibia = target_joint_angles_front[:,15]
joint_RHTarsus = target_joint_angles_front[:,16]


all_joint_angles = np.ones((target_num_steps, len(all_joint_angles[0]))) # 10000, 42
print(all_joint_angles.shape)

all_joint_angles[:,14] = joint_LHCoxa[:target_num_steps] # jusqu'à 10'000
all_joint_angles[:,15] = joint_LHCoxa_roll[:target_num_steps]
all_joint_angles[:,16] = joint_LHCoxa_yaw[:target_num_steps]
all_joint_angles[:,17] = joint_LHFemur[:target_num_steps]
all_joint_angles[:,18] = joint_LHFemur_roll[:target_num_steps]
all_joint_angles[:,19] = joint_LHTibia[:target_num_steps]
all_joint_angles[:,20] = joint_LHTarsus[:target_num_steps]
all_joint_angles[:,35] = joint_RHCoxa[:target_num_steps]
all_joint_angles[:,36] = joint_RHCoxa_roll[:target_num_steps]
all_joint_angles[:,37] = joint_RHCoxa_yaw[:target_num_steps]
all_joint_angles[:,38] = joint_RHFemur[:target_num_steps]
all_joint_angles[:,39] = joint_RHFemur_roll[:target_num_steps]
all_joint_angles[:,40] = joint_RHTibia[:target_num_steps]
all_joint_angles[:,41] = joint_RHTarsus[:target_num_steps]


#### Simulation

In [ ]:
# test_front = grooming_modules["foreleg"][:, :target_num_steps]
# abd_arr = 0.25*np.ones((5, 558))
# test_front = np.concatenate((test_front, abd_arr), axis=0)
# print(test_front.shape)
# print(nmf.arena)
# print(nmf.init_pose)


for i in trange(target_num_steps/2):
    for k in range (len(all_joint_angles[0])):
        if k == 9: # LH Tarsus
            joint_angles = all_joint_angles[k,i + np.pi/2]
        elif k == 41: # RH Tarsus
            joint_angles = all_joint_angles[k,i + np.pi/2]
        else:
            joint_angles = all_joint_angles[k,i]
    action = {"joints": joint_angles, "adhesion" : np.array([1,1,0,1,1,0])}
    #print("prog_action", action,'\n')
    nmf.step(action)
    nmf.render()

nmf.save_video("./outputs/main_groom.mp4")

In [ ]:
Video("./outputs/main_groom.mp4")